In [1]:
import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
!python -m spacy download uk_core_news_sm
!python -m spacy download ru_core_news_sm


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-18 20:00:27.325650: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.9 MB 1.2 MB/s 
     |████████████████████████████████| 5.0 MB 3.7 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 8.2 MB 22.4 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=df7725eb13269c565570f8698288a61c620a0ab3279ba3ae5c17e52ecd7066e8
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.lo

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
import numpy as np
nlp_u = spacy.load('uk_core_news_sm')
nlp_r = spacy.load('ru_core_news_sm')
prep_text = nlp_u('мій текст')
print(dir(prep_text))


['_', '__bytes__', '__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__ne__', '__new__', '__pyx_vtable__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '_bulk_merge', '_context', '_get_array_attrs', '_realloc', '_vector', '_vector_norm', 'cats', 'char_span', 'copy', 'count_by', 'doc', 'ents', 'extend_tensor', 'from_array', 'from_bytes', 'from_dict', 'from_disk', 'from_docs', 'from_json', 'get_extension', 'get_lca_matrix', 'has_annotation', 'has_extension', 'has_unknown_spaces', 'has_vector', 'is_nered', 'is_parsed', 'is_sentenced', 'is_tagged', 'lang', 'lang_', 'mem', 'noun_chunks', 'noun_chunks_iterator', 'remove_extension', 'retokenize', 'sentiment', 'sents', 'set_ents', 'set_extension', 'similarity', 'spans', 'tensor', 'text', 'te

In [4]:
df = pd.read_csv('../db.csv', index_col=['id'])
df.parsed_text = df.parsed_text.str.lower()
df.propaganda = df.propaganda.apply(lambda x: 0 if x == 'f' else 1)
df.parsed_text = df['parsed_text'].str.split(' ')
rus_df = df[df.language_code == 'rus']
ua_df = df[df.language_code == 'ua']
ua_df['parsed_text'] = ua_df['parsed_text'].apply(lambda x: np.average([nlp_u(val).vector_norm for val in x]))
rus_df['parsed_text'] = rus_df['parsed_text'].apply(lambda x: np.average([nlp_r(val).vector_norm for val in x]))
print(ua_df)
# new_df['parsed_text'] = new_df['parsed_text'].apply(lambda x: np.average([nlp_u(val).vector_norm for val in x]))

<ipython-input-4-a434e67e961e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ua_df['parsed_text'] = ua_df['parsed_text'].apply(lambda x: np.average([nlp_u(val).vector_norm for val in x]))


      parsed_text  propaganda language_code
id                                         
1147    10.145156           0            ua
1155    10.531172           0            ua
1167    10.625689           0            ua
1274    10.371606           0            ua
1197    10.712618           0            ua
...           ...         ...           ...
5182    10.706962           1            ua
5183    10.626012           1            ua
5185    10.271087           1            ua
5186    10.526261           1            ua
5187    10.649790           1            ua

[1269 rows x 3 columns]


<ipython-input-4-a434e67e961e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rus_df['parsed_text'] = rus_df['parsed_text'].apply(lambda x: np.average([nlp_r(val).vector_norm for val in x]))


In [5]:
new_df = pd.concat([rus_df, ua_df])
new_df['parsed_text_norm'] = new_df.parsed_text.apply(lambda x: (x - new_df.parsed_text.min())/(new_df.parsed_text.max() - new_df.parsed_text.min()))

new_df


,parsed_text,propaganda,language_code,parsed_text_norm
id,,,,
2,10.236153,1,rus,0.831579
4,10.078384,1,rus,0.818349
5,11.092068,1,rus,0.903350
6,10.300344,1,rus,0.836961
7,10.739521,1,rus,0.873788
...,...,...,...,...
5182,10.706962,1,ua,0.871058
5183,10.626012,1,ua,0.864270
5185,10.271087,1,ua,0.834508


In [6]:
model = LogisticRegression()
x, y = new_df['parsed_text'].values.reshape(-1, 1), new_df['propaganda'].values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)

In [7]:
model.fit(x_train, y_train)
model.score(x_test, y_test)

0.7155963302752294